<a href="https://colab.research.google.com/github/KodumuruRaja/IMBalanced-Balanced-data/blob/main/Imbalanced-Data-Over-and-Under-Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [ ]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [ ]:
# Load the data
file_path = Path('/content/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()
df
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

#df.head()

In [ ]:

# Convert string values into numerical ones using get_dummies method.
df1 = pd.get_dummies(data=df, columns=['home_ownership', 'verification_status','issue_d', 'pymnt_plan', \
                                      'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', \
                                      'debt_settlement_flag'])
df1

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,...,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500,0.1719,375.35,66000.0,low_risk,27.24,0,0,8,0,1609,14,10204.87,10204.87,355.29,355.29,295.13,60.16,0.0,0.0,0.0,375.35,0,1,0,178,38199,3,4,2,4,3.0,36590,59.0,1,2,1553,58.0,3700,1,...,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,0.0,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,1,0,1,1
1,25000,0.2000,929.09,105000.0,low_risk,20.23,0,0,17,1,18368,34,24293.13,24293.13,873.53,873.53,706.87,166.66,0.0,0.0,0.0,929.09,0,1,0,165,228119,3,3,2,3,4.0,42273,86.0,4,8,5197,61.0,50400,4,...,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,0.0,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,1,0,1,1
2,20000,0.2000,529.88,56000.0,low_risk,24.26,0,0,8,0,13247,21,19636.79,19636.79,485.44,485.44,363.21,122.23,0.0,0.0,0.0,529.88,0,1,0,0,45684,2,3,2,3,4.0,32437,75.0,2,3,2810,75.0,17500,0,...,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,0.0,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,1,0,1,1
3,10000,0.1640,353.55,92000.0,low_risk,31.44,0,1,10,1,17996,23,9719.34,9719.34,335.33,335.33,280.66,54.67,0.0,0.0,0.0,353.55,0,1,0,0,68784,0,4,1,3,9.0,50788,66.0,0,1,6360,69.0,23000,3,...,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,0.0,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,1,0,1,1
4,22000,0.1474,520.39,52000.0,low_risk,18.76,0,1,14,0,9091,28,21614.73,21614.73,484.36,484.36,385.27,99.09,0.0,0.0,0.0,520.39,0,1,0,0,177745,0,1,0,2,14.0,16828,84.0,1,1,3600,41.0,43100,2,...,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,1,0,0,1,0,0,0,0,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57588,35000,0.1033,1134.79,130000.0,low_risk,20.09,0,1,15,0,29775,21,32477.92,32477.92,3384.28,3384.28,2522.08,862.20,0.0,0.0,0.0,1134.79,0,1,0,0,74157,2,4,2,4,11.0,44382,69.0,2,4,6562,63.0,53600,1,...,7.0,9.0,8.0,9.0,7.0,11.0,14.0,9.0,15.0,0.0,0.0,0.0,4.0,100.0,42.9,0.0,0.0,117619.0,74157.0,38600.0,64019.0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1
57589,24000,0.0819,488.82,147000.0,low_risk,13.26,0,0,8,1,15241,19,23018.27,23018.27,1455.54,1455.54,981.73,473.81,0.0,0.0,0.0,488.82,0,1,0,0,520047,1,3,1,1,11.0,100138,58.0,1,2,8721,40.0,46400,2,...,3.0,3.0,4.0,5.0,12.0,4.0,5.0,3.0,8.0,0.0,0.0,0.0,2.0,100.0,0.0,1.0,0.0,571938.0,115379.0,46400.0,108538.0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,1
57590,25000,0.0646,765.77,175000.0,low_risk,

In [ ]:
# Create our features

X = df1.copy()
X = X.drop("loan_status", axis=1)
X.head()


# Create our target
y = df1["loan_status"].values
y

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [ ]:
x.describe

AttributeError: ignored

In [ ]:
type(y[0])

str

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(43194, 95)
(14399, 95)
(43194,)
(14399,)


# 1) OverSampling

# i) **Naive RandomOverSampling**

In [ ]:
#Viewing count and imbalance
Counter(y_train)

Counter({'high_risk': 184, 'low_risk': 43010})

In [ ]:
y_train

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [ ]:
# Resample the training# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 43010, 'low_risk': 43010})

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7248473464750913

In [ ]:

# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  43,   11],
       [4972, 9373]])

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.80      0.65      0.02      0.72      0.53        54
   low_risk       1.00      0.65      0.80      0.79      0.72      0.51     14345

avg / total       1.00      0.65      0.80      0.79      0.72      0.51     14399



# ii) **SMOTE OverSampling**

In [ ]:

# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

In [ ]:

# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7518027961736571

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   42,    12],
       [ 3933, 10412]])

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.78      0.73      0.02      0.75      0.57        54
   low_risk       1.00      0.73      0.78      0.84      0.75      0.56     14345

avg / total       1.00      0.73      0.78      0.84      0.75      0.56     14399



# **UnderSampling**

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 184, 'low_risk': 184})

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.604149077624156

In [ ]:

# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  34,   20],
       [6044, 8301]])

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.58      0.01      0.60      0.37        54
   low_risk       1.00      0.58      0.63      0.73      0.60      0.36     14345

avg / total       0.99      0.58      0.63      0.73      0.60      0.36     14399



# **Combination (Over and Under) Sampling**

In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [ ]:
Counter(y_resampled)

Counter({'high_risk': 57350, 'low_risk': 52476})

In [ ]:

# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7313756244916928

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  45,    9],
       [5316, 9029]])

In [ ]:

# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.83      0.63      0.02      0.72      0.54        54
   low_risk       1.00      0.63      0.83      0.77      0.72      0.51     14345

avg / total       1.00      0.63      0.83      0.77      0.72      0.51     14399

